In [145]:
import json

import funcy
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from skmultilearn.model_selection import iterative_train_test_split

from super_pycocotools.coco import SuperCOCO

In [105]:
super_ann_file = './datasets.json'

#initialize COCO api
coco = SuperCOCO(super_ann_file)

processing LogoDet
train data
loading annotations into memory...
Done (t=0.97s)
creating index...
index created!


In [106]:
len(coco.LogoDet.train.getImgIds())

158654

In [158]:
annotation_ids = coco.LogoDet.train.getAnnIds()
annotations = coco.LogoDet.train.loadAnns(annotation_ids)

In [ ]:
annotation_ids = funcy.lmap(lambda a: int(a['category_id']), annotations)
annotation_ids = funcy.lremove(lambda i: annotation_ids.count(i) <= 30, annotation_ids)
annotations = funcy.lremove(lambda a: a['category_id'] not in annotation_ids, annotations)

In [107]:
category_ids = coco.LogoDet.train.getCatIds()
categories = coco.LogoDet.train.loadCats(category_ids)

In [ ]:
categories = funcy.lremove(lambda c: c['id'] not in annotation_ids, categories)

In [140]:
logo_split = {}
for ann_id in annotation_ids:
    annotation = coco.LogoDet.train.loadAnns(ann_id)[0]
    cat_id = annotation['category_id']
    im_id = annotation['image_id']
    if cat_id not in logo_split:
        logo_split[cat_id] = []
    logo_split[cat_id].append(im_id)

In [146]:
train = []
test = []
for cat_id, im_id in logo_split.items():
    x_train, x_test = train_test_split(im_id, train_size=0.8)
    train += list(x_train)
    test += list(x_test)

In [154]:
image_ids = coco.LogoDet.train.getImgIds()
images = coco.LogoDet.train.loadImgs(image_ids)

In [157]:
train_annotations = funcy.lfilter(lambda a: int(a['image_id']) in train, annotations)
train_images = funcy.lfilter(lambda i: int(i['id']) in train, images)

test_annotations = funcy.lfilter(lambda a: int(a['image_id']) in test, annotations)
test_images = funcy.lfilter(lambda i: int(i['id']) in test, images)

In [ ]:
trainjson = "./instances_train2017.json"

with open(trainjson, 'wt', encoding='UTF-8') as coco:
    json.dump({'images': train_images, 'annotations': train_annotations, 'categories': categories}, coco, indent=2, sort_keys=True)

testjson = "./instances_test2017.json"

with open(testjson, 'wt', encoding='UTF-8') as coco:
    json.dump({'images': test_images, 'annotations': test_annotations, 'categories': categories}, coco, indent=2, sort_keys=True)